https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_split.ipynb

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [71]:
transaction = pd.read_csv('book_transactions.csv', encoding='cp949')

In [72]:
data = transaction[['회원번호', '책제목']]
ids = data.groupby('회원번호').책제목.count().reset_index().query("책제목 > 5").회원번호.unique()
data = data.query('회원번호 in @ids')

id2user = dict(enumerate(data.회원번호.unique()))
user2id = {j:i for i, j in id2user.items()}
id2book = dict(enumerate(data.책제목.unique()))
book2id = {j:i for i, j in id2book.items()}
#id2cat = dict(enumerate(data.카테고리.unique()))
#cat2id = {j:i for i, j in id2cat.items()}

data.회원번호 = data.회원번호.map(lambda x: user2id[x])
data.책제목 = data.책제목.map(lambda x: book2id[x])
data = data.drop_duplicates()

In [73]:
#data_cat = transaction.groupby('책제목')['카테고리'].unique().map(lambda x: x[0]).reset_index()

In [74]:
#data_cat['책제목'] = data_cat.책제목.map(lambda x: book2id[x])
#data_cat['카테고리'] = data_cat.카테고리.map(lambda x: cat2id[x])

In [75]:
#data_cat = dict(data_cat.values)

In [76]:
data.to_csv('main_data_yes24.csv', index=False)

In [77]:
#data_cat = transaction[['회원번호', '카테고리']]
#data_cat.회원번호 = data_cat.회원번호.map(lambda x: user2id[x])
#data_cat.카테고리 = data_cat.카테고리.map(lambda x: cat2id[x])
#data_cat = data_cat.drop_duplicates()

In [78]:
data_tr = pd.read_csv('main_data_yes24.csv')

In [79]:
data_tr.shape

(203652, 2)

In [80]:
train_data = data_tr.copy()

In [81]:
items = data.groupby('회원번호')['책제목'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = train_data.append(pd.DataFrame(test_data, columns=['회원번호', '책제목'])).drop_duplicates(keep=False)

In [82]:
train_data.to_csv('main_data_train_yes24.csv', index=False)

In [83]:
item_num = data_tr.책제목.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [84]:
np.save('main_data_test_yes24.npy', res.reset_index().values, allow_pickle=True)

In [85]:
test_data_raw = np.load('main_data_test_yes24.npy', allow_pickle=True)

In [86]:
test_data = []
for data in test_data_raw:
    u = data[0]
    i = data[1]
    for item in i:
        test_data.append([u, int(item)])

In [87]:
print(data_tr.회원번호.nunique(), data_tr.책제목.nunique())

12124 60210


ml-1m

In [154]:
column_names = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv("./ml-1m/movies.dat", sep = "::", names = column_names, encoding='latin-1', engine='python')

column_names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users = pd.read_csv("./ml-1m/users.dat", sep = "::", names = column_names, engine='python')

column_names = ['User_ID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv("./ml-1m/ratings.dat", sep = "::", names = column_names, engine='python')
ratings.head()

,User_ID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
id2user = dict(enumerate(users.User_ID.unique()))
user2id = {j:i for i, j in id2user.items()}

id2item = dict(enumerate(movies.MovieID.unique()))
item2id = {j:i for i, j in id2item.items()}

In [4]:
ratings['User_ID'] = ratings['User_ID'].map(lambda x: user2id[x]) 
ratings['MovieID'] = ratings['MovieID'].map(lambda x: item2id[x]) 
users['User_ID'] = users['User_ID'].map(lambda x: user2id[x]) 
movies['MovieID'] = movies['MovieID'].map(lambda x: item2id[x]) 

In [ ]:
# Aux

In [8]:
users['aux'] = users['Occupation']

In [10]:
users['aux'] = users[['Gender', 'Age', 'Occupation']].apply(lambda x: ','.join(map(lambda x: str(x),x)), axis=1)

In [ ]:
movies[['MovieID', 'Genres']].to_csv('ml_movies.csv', index=False)

In [161]:
movies.Genres = movies.Genres.map(lambda x: x.split('|')[0])
movies[['MovieID', 'Genres']].to_csv('ml_movies2.csv', index=False)

In [9]:
users[['User_ID', 'aux']].to_csv('ml_users2.csv', index=False)

In [164]:
movies[['MovieID', 'Genres']].Genres.nunique()#, users[['User_ID', 'aux']].aux.nunique()

18

In [15]:
ratings = ratings.sort_values(['User_ID', 'Timestamp'])[['User_ID', 'MovieID']].reset_index(drop=True)

In [16]:
ratings['User_ID'].value_counts().describe()

count    6040.000000
mean      165.597517
std       192.747029
min        20.000000
25%        44.000000
50%        96.000000
75%       208.000000
max      2314.000000
Name: User_ID, dtype: float64

In [17]:
ratings_copy = ratings.copy()

In [18]:
items = ratings.groupby('User_ID')['MovieID'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User_ID', 'MovieID'])).drop_duplicates(keep=False)

In [19]:
train_data

,User_ID,MovieID
0,0,3117
1,0,1250
2,0,1672
3,0,1009
4,0,2271
...,...,...
1000203,6039,229
1000204,6039,2848
1000205,6039,1852
1000206,6039,1726


In [20]:
test_data

array([[   0,   47],
       [   1, 1848],
       [   2, 2012],
       ...,
       [6037, 1166],
       [6038, 1234],
       [6039, 1203]], dtype=int64)

In [21]:
train_data.to_csv('main_data_train_ml.csv', index=False)

In [22]:
item_num = ratings_copy.MovieID.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [23]:
np.save('main_data_test_ml.npy', res.reset_index().values, allow_pickle=True)

In [24]:
ratings.User_ID.max(), ratings.MovieID.max()

(6039, 3882)

Amazon Book Data

In [128]:
rating_am = pd.read_csv('./amazon_book_dataset/Ratings.csv')
users_am = pd.read_csv('./amazon_book_dataset/Users.csv')
items_am = pd.read_csv('./amazon_book_dataset/Books.csv')

C:\Users\KwonYujin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [129]:
rating_am.groupby('User-ID').ISBN.count().describe()

count    105283.000000
mean         10.920851
std          90.562825
min           1.000000
25%           1.000000
50%           1.000000
75%           4.000000
max       13602.000000
Name: ISBN, dtype: float64

In [130]:
ids = rating_am.groupby('User-ID').ISBN.count().reset_index().query('ISBN > 5')['User-ID'].values
rating_am = rating_am.query('`User-ID` in @ids').reset_index(drop=True)
users_am = users_am.query('`User-ID` in @ids').reset_index(drop=True)

books = rating_am['ISBN'].unique()
items_am = items_am.query('ISBN in @books').reset_index(drop=True)

In [131]:
id2user = dict(enumerate(users_am['User-ID'].unique()))
user2id = {j:i for i, j in id2user.items()}

id2item = dict(enumerate(rating_am.ISBN.unique()))
item2id = {j:i for i, j in id2item.items()}

In [132]:
rating_am['User-ID'] = rating_am['User-ID'].map(lambda x: user2id[x]) 
rating_am['ISBN'] = rating_am['ISBN'].map(lambda x: item2id[x]) 
users_am['User-ID'] = users_am['User-ID'].map(lambda x: user2id[x]) 
items_am['ISBN'] = items_am['ISBN'].map(lambda x: item2id[x]) 

In [93]:
ratings_copy = rating_am.copy()[['User-ID', 'ISBN']]

In [94]:
items = rating_am.groupby('User-ID')['ISBN'].unique()
test_data = items.map(lambda x: x[-1]).reset_index().values
train_data = ratings_copy.append(pd.DataFrame(test_data, columns=['User-ID', 'ISBN'])).drop_duplicates(keep=False)

In [97]:
test_data

array([[    0,  8158],
       [    1,  8164],
       [    2,  8171],
       ...,
       [19714,  8113],
       [19715,  8136],
       [19716,  1758]], dtype=int64)

In [98]:
train_data.to_csv('main_data_train_am.csv', index=False)

In [99]:
item_num = ratings_copy.ISBN.nunique()
item_list = set(range(item_num))
def noninteracted(x):
    lst = list(item_list - set(x))
    sampled = np.random.choice(lst, 99)
    result = x[-1].reshape(1).tolist() + sampled.tolist()
    return result
res = items.map(noninteracted)

In [103]:
np.save('main_data_test_am.npy', res.reset_index().values, allow_pickle=True)

In [106]:
rating_am['User-ID'].max(), rating_am.ISBN.max()

(19716, 316098)

In [133]:
users_am.Age.fillna(users_am.Age.mode()[0], inplace= True)

In [134]:
users_am.Age.describe()

count    19717.000000
mean        33.951057
std         12.807987
min          0.000000
25%         29.000000
50%         29.000000
75%         38.000000
max        244.000000
Name: Age, dtype: float64

In [135]:
def age(x):
    if x < 10:
        return '10대 미만'
    else:
        return str(x)[0]+'0대'

users_am['Age'] = users_am.Age.map(age)

In [137]:
users_am[['User-ID', 'Age']]

,User-ID,Age
0,0,20대
1,1,20대
2,2,20대
3,3,20대
4,4,40대
...,...,...
19712,19712,30대
19713,19713,10대
19714,19714,20대
19715,19715,30대


In [149]:
items_am[['ISBN', 'Publisher']]

,ISBN,Publisher
0,8141,HarperFlamingo Canada
1,8142,HarperPerennial
2,8143,Farrar Straus Giroux
3,8144,W. W. Norton &amp; Company
4,8145,Putnam Pub Group
...,...,...
256721,315861,Random House Childrens Pub (Mm)
256722,316009,Dutton Books
256723,316064,HarperSanFrancisco
256724,316066,Oxford University Press


In [152]:
items_am

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,8141,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
1,8142,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
2,8143,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
3,8144,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
4,8145,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...
...,...,...,...,...,...,...,...,...
256721,315861,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
256722,316009,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
256723,316064,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
256724,316066,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [151]:
items_am.groupby('Publisher')['ISBN'].count().describe()

count    15575.000000
mean        16.483082
std        129.512730
min          1.000000
25%          1.000000
50%          1.000000
75%          4.000000
max       7506.000000
Name: ISBN, dtype: float64

In [153]:
items_am.groupby('Publisher')['Year-Of-Publication'].count().describe()

count    15575.000000
mean        16.483082
std        129.512730
min          1.000000
25%          1.000000
50%          1.000000
75%          4.000000
max       7506.000000
Name: Year-Of-Publication, dtype: float64